In [7]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=200, device='cpu'):
        super(PositionalEncoding, self).__init__()
        self.pos_table = self._get_sinusoid_encoding_table(max_len, d_model, device)

    def _get_sinusoid_encoding_table(self, max_seq_len, d_hid, device):
        position = torch.arange(0, max_seq_len, device=device).float().unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_hid, 2, device=device).float() * -(torch.log(torch.tensor(10000.0)) / d_hid))
        
        sinusoid_table = torch.zeros(max_seq_len, d_hid, device=device)
        sinusoid_table[:, 0::2] = torch.sin(position * div_term)
        sinusoid_table[:, 1::2] = torch.cos(position * div_term)

        return sinusoid_table.unsqueeze(0)

    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()

# Initialize Positional Encoding
d_model = 8
max_len = 10
pos_enc = PositionalEncoding(d_model, max_len)

# Generate example data
batch_size = 1
seq_len = 5
x = torch.rand(batch_size, seq_len, d_model)

print("Original Input:\n", x)
print("input shape:", x.shape)

# Apply Positional Encoding
x_encoded = pos_enc(x)

print("\nPositional Encoded Input:\n", x_encoded)
print("output shape", x_encoded.shape)

Original Input:
 tensor([[[0.1498, 0.2171, 0.2090, 0.7003, 0.8706, 0.0539, 0.2485, 0.6293,
          0.6528, 0.8268, 0.8233, 0.3336, 0.3325, 0.4685, 0.3345, 0.4057,
          0.7719, 0.0992, 0.8649, 0.5906],
         [0.5077, 0.0263, 0.6695, 0.5536, 0.5568, 0.1310, 0.1975, 0.3713,
          0.0521, 0.2612, 0.4569, 0.7310, 0.5098, 0.5987, 0.3814, 0.5719,
          0.2310, 0.0743, 0.5682, 0.8773],
         [0.8124, 0.9283, 0.1257, 0.4863, 0.8155, 0.9625, 0.4131, 0.5422,
          0.7215, 0.5551, 0.6208, 0.2217, 0.5136, 0.9822, 0.7423, 0.4047,
          0.7964, 0.5500, 0.7129, 0.6685],
         [0.6969, 0.7506, 0.5955, 0.0191, 0.7301, 0.6475, 0.5636, 0.5898,
          0.5288, 0.1977, 0.0337, 0.8739, 0.6883, 0.5358, 0.3201, 0.5920,
          0.2005, 0.3696, 0.8790, 0.2149],
         [0.0141, 0.0640, 0.8949, 0.9949, 0.4060, 0.1965, 0.0362, 0.4216,
          0.0655, 0.9540, 0.2774, 0.3447, 0.9953, 0.6674, 0.3529, 0.6556,
          0.7959, 0.7898, 0.6810, 0.4014]]])
input shape: torch.Size([1

RuntimeError: The size of tensor a (20) must match the size of tensor b (8) at non-singleton dimension 2